In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

pd.set_option('display.max_columns', None)

In [ ]:
train = pd.read_csv('../input/pubg-finish-placement-prediction/train_V2.csv')
test = pd.read_csv('../input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
train.info()

* groupId - ID для идентификации группы в матче. Если одна и та же группа игроков играет в разных матчах, у них будет разный groupId каждый раз.
* matchId - ID для идентификации матча.
* assists - Количество вражеских игроков, поврежденных этим игроком и убитых товарищами по команде.
* boosts - Количество использованных бустеров.
* damageDealt - Общий урон. Примечание: урон по самому себе вычитается.
* DBNOs - Количество игроков, которые были кнокнуты.
* headshotKills - Количество убитых игроков выстрелами в голову.
* heals - Количество использованных лечащих предметов.
* killPlace - Место, которое игрок занял в матче.
* killPoints - Очки, основанные на рейтинге игроков. (Думайте об этом как о рейтинге Эло, где важны только убийства).
* kills - Количество убийств.
* killStreaks - Максимальное количество вражеских игроков, убитых за короткое время.
* longestKill - Наибольшее расстояние между игроком и игроком, убитым в момент смерти. Это может вводить в заблуждение, так как отстранение игрока от игры и отъезд могут привести к большой статистике.
* maxPlace - Худшее место у нас есть данные в матче. Это может не совпадать с numGroups, так как иногда данные пропускаются по местам размещения.
* numGroups - Количество групп, по которым у нас есть данные в матче.
* revives - Сколько раз этот игрок восстанавливал товарищей по команде.
* rideDistance - Общее пройденное расстояние в транспортных средствах, измеренное в метрах.
* roadKills - Количество убийств в автомобиле.
* swimDistance - Общее расстояние, пройденное плаванием в метрах.
* teamKills - Сколько раз этот игрок убивал товарища по команде.
* vehicleDestroys - Количество уничтоженных автомобилей.
* walkDistance - Общее пройденное расстояние пешком в метрах.
* weaponsAcquired - Количество поднятого оружия.
* winPoints - Победный внешний рейтинг игрока. (Думайте об этом как о рейтинге Эло, где важен только выигрыш).
* winPlacePerc - Цель прогнозирования. Это выигрышное размещение в процентилях, где 1 соответствует 1-му месту, а 0 соответствует последнему месту в матче. Он рассчитывается по maxPlace, а не по numGroups, поэтому в совпадении могут быть пропущены фрагменты.


In [ ]:
train.head()

In [ ]:
train.describe()

## Очистка датасета

In [ ]:
train[train['winPlacePerc'].isnull()]

In [ ]:
train.drop(2744604, inplace=True)

# Выявление аномалий

## Убийства без перемещения

In [ ]:
# Создание признака totalDistance
train['totalDistance'] = train['rideDistance'] + train['walkDistance'] + train['swimDistance']
# Создание признака killsWithoutMoving
train['killsWithoutMoving'] = ((train['kills'] > 0) & (train['totalDistance'] == 0))

## Соотношение числа убийств в голову и количества убийств

In [ ]:
# Создание признака headshot_rate
train['headshot_rate'] = train['headshotKills'] / train['kills']
train['headshot_rate'] = train['headshot_rate'].fillna(0)

## Убийства без перемещения

In [ ]:
display(train[train['killsWithoutMoving'] == True].shape)
train[train['killsWithoutMoving'] == True].head(10)

In [ ]:
train.drop(train[train['killsWithoutMoving'] == True].index, inplace=True)
train.drop('killsWithoutMoving', axis=1, inplace=True)

## Аномальные убийтва на транспорте

In [ ]:
plt.figure(figsize=(12, 4))
sns.distplot(train['roadKills'], bins=50)
plt.show()

In [ ]:
train[train['roadKills'] >= 10][['Id', 'kills', 'roadKills', 'rideDistance', 'walkDistance']]

In [ ]:
train.drop(train[train['roadKills'] >= 10].index, inplace=True)

## Аномальное количество убийств

In [ ]:
plt.figure(figsize=(12,4))
sns.countplot(data=train, x=train['kills']).set_title('Kills')
plt.show()

In [ ]:
train[(train['kills'] > 30) & (train['totalDistance'] < 2000)][['kills','totalDistance']]

In [ ]:
train.drop(train[(train['kills'] > 30) & (train['totalDistance'] < 2000)].index, inplace=True)
train.drop('totalDistance', axis=1, inplace=True)

## Аномальные убийства в голову

In [ ]:
plt.figure(figsize=(15,4))
sns.distplot(train['headshot_rate'], bins=10)
plt.show()

In [ ]:
display(train[(train['headshot_rate'] == 1) & (train['kills'] >= 10)].shape)
train[(train['headshot_rate'] == 1) & (train['kills'] >= 10)].head(10)

In [ ]:
train.drop(train[(train['headshot_rate'] == 1) & (train['kills'] >= 10)].index, inplace=True)

## Аномальные убийства с большой дистанции

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['longestKill'], bins=10)
plt.show()

In [ ]:
display(train[train['longestKill'] >= 1000].shape)
train[train['longestKill'] >= 1000].head(10)

In [ ]:
train.drop(train[train['longestKill'] >= 1000].index, inplace=True)

## Аномальные перемещения

### walkDistance

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train[train['walkDistance'] >= 10000]['walkDistance'], bins=10)
plt.show()

In [ ]:
display(train[train['walkDistance'] >= 10000].shape)
train[train['walkDistance'] >= 10000].head(10)

In [ ]:
v = 22.5 / 3.6 # км/ч в м/с

In [ ]:
train[train['walkDistance'] >= v * train['matchDuration']]

In [ ]:
train.drop(train[train['walkDistance'] >= v * train['matchDuration']].index, inplace=True)

### rideDistance

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['rideDistance'], bins=10)
plt.show()

In [ ]:
display(train[train['rideDistance'] >= 20000].shape)
train[train['rideDistance'] >= 20000].head(10)

In [ ]:
v = 100 / 3.6 # км/ч в м/с
train[train['rideDistance'] >= v * train['matchDuration']]

In [ ]:
train.drop(train[train['rideDistance'] >= 20000].index, inplace=True)

### swimDistance

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['swimDistance'], bins=10)
plt.show()

In [ ]:
train[train['swimDistance'] >= 2000]

In [ ]:
train.drop(train[train['swimDistance'] >= 2000].index, inplace=True)

## Аномалии с предметами

### weaponsAcquired

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['weaponsAcquired'], bins=100)
plt.show()

In [ ]:
display(train[train['weaponsAcquired'] >= 50].shape)
train[train['weaponsAcquired'] >= 50].head(10)

In [ ]:
train.drop(train[train['weaponsAcquired'] >= 50].index, inplace=True)

### heals

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['heals'], bins=10)
plt.show()

In [ ]:
display(train[train['heals'] >= 40].shape)
train[train['heals'] >= 40].head(10)

In [ ]:
train.drop(train[train['heals'] >= 40].index, inplace=True)

# Корреляция между признаками

In [ ]:
f,ax = plt.subplots(figsize=(15, 15))
sns.heatmap(train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

# Конструирование признаков

## Начальное количество игроков в матче

In [ ]:
train['playersJoined'] = train.groupby('matchId')['matchId'].transform('count')
plt.figure(figsize=(15,10))
sns.countplot(train[train['playersJoined']>=75]['playersJoined'])
plt.title('playersJoined')
plt.show()

## Нормализация признаков
Используя признак количества игроков в матче, мы можем нормализовать некоторые признаки: 
1. kills
2. damageDealt
3. maxPlace
4. matchDuration

In [ ]:
# создание нормализованных признаков
train['killsNorm'] = train['kills']*((100-train['playersJoined'])/100 + 1)
train['damageDealtNorm'] = train['damageDealt']*((100-train['playersJoined'])/100 + 1)
train['maxPlaceNorm'] = train['maxPlace']*((100-train['playersJoined'])/100 + 1)
train['matchDurationNorm'] = train['matchDuration']*((100-train['playersJoined'])/100 + 1)
# сравнение нормализованных признаков
to_show = ['Id', 'kills','killsNorm','damageDealt', 'damageDealtNorm', 'maxPlace', 'maxPlaceNorm', 'matchDuration', 'matchDurationNorm', 'playersJoined']
train[to_show][0:11]

# Типы матча

In [ ]:
# Turn groupId and match Id into categorical types
train['groupId'] = train['groupId'].astype('category')
train['matchId'] = train['matchId'].astype('category')

# Get category coding for groupId and matchID
train['groupId_cat'] = train['groupId'].cat.codes
train['matchId_cat'] = train['matchId'].cat.codes

# Get rid of old columns
train.drop(columns=['groupId', 'matchId'], inplace=True)

# Lets take a look at our newly created features
train[['groupId_cat', 'matchId_cat']].head()

In [ ]:
train.drop(columns = ['Id'], inplace=True)

In [ ]:
train['matchType'].value_counts().plot.barh()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()

mapped_education = pd.Series(label_encoder.fit_transform(train['matchType']))
mapped_education.value_counts().plot.barh()
print(dict(enumerate(label_encoder.classes_)))

In [ ]:
# Label Encoding
categorical_columns = train.columns[train.dtypes == 'object'].union(['matchType'])
for column in categorical_columns:
    train[column] = label_encoder.fit_transform(train[column])
train.head(10)

# Машинное обучение

In [ ]:
target = 'winPlacePerc'
features = list(train.columns)


y_train = np.array(train[target])
features.remove(target)
x_train = train[features]

print(x_train.shape,y_train.shape)

In [ ]:
test['headshot_rate'] = test['headshotKills'] / test['kills']
test['headshot_rate'] = test['headshot_rate'].fillna(0)
test['playersJoined'] = test.groupby('matchId')['matchId'].transform('count')
test['killsNorm'] = test['kills']*((100-test['playersJoined'])/100 + 1)
test['damageDealtNorm'] = test['damageDealt']*((100-test['playersJoined'])/100 + 1)
test['maxPlaceNorm'] = test['maxPlace']*((100-train['playersJoined'])/100 + 1)
test['matchDurationNorm'] = test['matchDuration']*((100-test['playersJoined'])/100 + 1)
test['healsandboosts'] = test['heals'] + test['boosts']

# Turn groupId and match Id into categorical types
test['groupId'] = test['groupId'].astype('category')
test['matchId'] = test['matchId'].astype('category')

# Get category coding for groupId and matchID
test['groupId_cat'] = test['groupId'].cat.codes
test['matchId_cat'] = test['matchId'].cat.codes


categorical_columns = test.columns[test.dtypes == 'object'].union(['matchType'])
for column in categorical_columns:
    test[column] = label_encoder.fit_transform(test[column])

# Remove irrelevant features from the test set
x_test = test[features].copy()

# Fill NaN with 0 (temporary)
x_test.fillna(0, inplace=True)

print(x_test.shape)

In [ ]:
x_test.head()

Разделим данные

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state = 1)

## Random Forest

Создаем Random Forest Regressor

In [ ]:
rf = RandomForestRegressor(n_estimators = 80, min_samples_leaf = 3, max_depth = 26, max_features = 0.5,
                          n_jobs = -1)

In [ ]:
x_train.head()

In [ ]:
x_train.shape

In [ ]:
%%time
rf.fit(x_train, y_train)

In [ ]:
print('mae train: ', mean_absolute_error(rf.predict(x_train), y_train))
print('mae val: ', mean_absolute_error(rf.predict(x_val), y_val))

In [ ]:
%%time
pred = rf.predict(x_test)
test['winPlacePerc'] = pred
submission = test[['Id', 'winPlacePerc']]
submission.to_csv('submission.csv', index=False)